In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

Spark = SparkSession.builder.appName("Sanfranciso").getOrCreate()

In [2]:
Spark

# 1. Read the sanfranciso fire calls and to pandas

In [33]:
sf_df=Spark.read.csv("sf-fire-calls.csv",header=True, inferSchema=True)
sf_df.toPandas()
#to view the ouput in the table

,CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,...,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay
0,20110016,T13,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:44 AM,2000 Block of CALIFORNIA ST,SF,...,None,1,TRUCK,2.0,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T13,2.950000
1,20110022,M17,2003241,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 03:01:18 AM,0 Block of SILVERVIEW DR,SF,...,None,1,MEDIC,1.0,10,10,Bayview Hunters Point,"(37.7337623673897, -122.396113802632)",020110022-M17,4.700000
2,20110023,M41,2003242,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:39:50 AM,MARKET ST/MCALLISTER ST,SF,...,None,1,MEDIC,2.0,3,6,Tenderloin,"(37.7811772186856, -122.411699931232)",020110023-M41,2.433333
3,20110032,E11,2003250,Vehicle Fire,01/11/2002,01/10/2002,Other,01/11/2002 04:16:46 AM,APPLETON AV/MISSION ST,SF,...,None,1,ENGINE,1.0,6,9,Bernal Heights,"(37.7388432849018, -122.423948785199)",020110032-E11,1.500000
4,20110043,B04,2003259,Alarms,01/11/2002,01/10/2002,Other,01/11/2002 06:01:58 AM,1400 Block of SUTTER ST,SF,...,None,1,CHIEF,2.0,4,2,Western Addition,"(37.7872890372638, -122.424236212664)",020110043-B04,3.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,183034235,T08,18127270,Structure Fire,10/30/2018,10/30/2018,Fire,10/30/2018 10:04:27 PM,700 Block of LONG BRIDGE ST,San Francisco,...,Alarm,1,TRUCK,3.0,3,6,Mission Bay,"(37.77139049567131, -122.39416141919062)",183034235-T08,2.483333
175292,183034238,86,18127271,Medical Incident,10/30/2018,10/30/2018,Code 2 Transport,10/30/2018 11:18:37 PM,300 Block of WILLIAMS AVE,San Francisco,...,Potentially Life-Threatening,1,MEDIC,2.0,10,10,Bayview Hunters Point,"(37.73017420329081, -122.39943089033613)",183034238-86,1.950000
175293,183034268,E08,18127272,Medical Incident,10/30/2018,10/30/2018,Code 2 Transport,10/30/2018 10:26:01 PM,1100 Block of 4TH ST,San Francisco,...,Potentially Life-Threatening,1,ENGINE,1.0,3,6,Mission Bay,"(37.77389059449051, -122.3915765620042)",183034268-E08,4.416666
175294,183034485,KM03,18127291,Medical Incident,10/30/2018,10/30/2018,Code 2 Transport,10/31/2018 12:52:43 AM,800 Block of HAIGHT ST,San Francisco,...,Non Life-threatening,1,PRIVATE,1.0,5,5,Haight Ashbury,"(37.77131921762313, -122.43619838315635)",183034485-KM03,3.566667


# 2. show the columns

In [8]:
sf_df.columns

['CallNumber',
 'UnitID',
 'IncidentNumber',
 'CallType',
 'CallDate',
 'WatchDate',
 'CallFinalDisposition',
 'AvailableDtTm',
 'Address',
 'City',
 'Zipcode',
 'Battalion',
 'StationArea',
 'Box',
 'OriginalPriority',
 'Priority',
 'FinalPriority',
 'ALSUnit',
 'CallTypeGroup',
 'NumAlarms',
 'UnitType',
 'UnitSequenceInCallDispatch',
 'FirePreventionDistrict',
 'SupervisorDistrict',
 'Neighborhood',
 'Location',
 'RowID',
 'Delay']

## 3. Print the schema of the dataset

In [9]:
sf_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

# Questions
 1. What are  all the different types of fire in the calls type

In [10]:

sf_1= sf_df.select("CallType")
fi_sf= sf_1.filter((sf_1["CallType"]).like("%Fire%")).groupBy("CallType").count()
fi_sf.show()

+--------------+-----+
|      CallType|count|
+--------------+-----+
|   Marine Fire|   14|
|  Vehicle Fire|  854|
|  Outside Fire| 2094|
|Structure Fire|23319|
+--------------+-----+



# 
2. How many fire calls in 2018 were categorized as "medical incidents"?

In [11]:

sf_2 = sf_df.withColumn("CallYear", year(to_date(sf_df["CallDate"], 'MM/dd/yyyy')))
fi_2018= sf_2.filter((sf_2["CallYear"]==2018))
res_1=fi_2018.filter((sf_2["CallType"]=="Medical Incident"))
res_1.groupBy("CallType","CallYear").count().show()

+----------------+--------+-----+
|        CallType|CallYear|count|
+----------------+--------+-----+
|Medical Incident|    2018| 7004|
+----------------+--------+-----+



  3. What were all the different types of fire calls  in 2018?

In [12]:

final=fi_2018.groupBy("CallType").count().orderBy("count", ascending=False).alias("TotalCalls")
final.show()

+--------------------+-----+
|            CallType|count|
+--------------------+-----+
|    Medical Incident| 7004|
|              Alarms| 1144|
|      Structure Fire|  906|
|   Traffic Collision|  433|
|        Outside Fire|  153|
|               Other|  114|
|Citizen Assist / ...|  113|
|Gas Leak (Natural...|   69|
|        Water Rescue|   43|
|Elevator / Escala...|   36|
|   Electrical Hazard|   30|
|        Vehicle Fire|   28|
|Smoke Investigati...|   28|
|Odor (Strange / U...|   10|
|          Fuel Spill|   10|
|              HazMat|    5|
|Train / Rail Inci...|    5|
|  Suspicious Package|    3|
|       Assist Police|    1|
|           Explosion|    1|
+--------------------+-----+



# 
4. Which neighborhood in San Francisco generated the most fire calls in 2018?

In [13]:

sf_4=fi_2018.groupBy(sf_2["Neighborhood"]).count()
sf_4.orderBy("count",ascending=False).show(5)

+--------------------+-----+
|        Neighborhood|count|
+--------------------+-----+
|          Tenderloin| 1393|
|     South of Market| 1053|
|             Mission|  913|
|Financial Distric...|  772|
|Bayview Hunters P...|  522|
+--------------------+-----+
only showing top 5 rows



#
5. Which neighborhoods had the worst response times to fire calls in 2018?

In [15]:

sf_5=fi_2018.groupBy(sf_2["Neighborhood"]).agg(max(sf_2["Delay"])).alias("Worst")
sf_5.orderBy("max(Delay)",ascending= False).show()

+--------------------+----------+
|        Neighborhood|max(Delay)|
+--------------------+----------+
|           Chinatown| 491.26666|
|Financial Distric...| 406.63333|
|          Tenderloin| 340.48334|
|      Haight Ashbury| 175.86667|
|Bayview Hunters P...|     155.8|
|     Pacific Heights| 129.01666|
|        Potrero Hill|     109.8|
|        Inner Sunset| 106.13333|
|     South of Market|  94.71667|
|      Inner Richmond| 90.433334|
|           Excelsior|  83.76667|
| Castro/Upper Market|  74.13333|
|    Western Addition| 67.916664|
|            Nob Hill|     67.45|
|             Mission| 54.666668|
|    Presidio Heights| 52.883335|
|       Outer Mission| 43.383335|
|  West of Twin Peaks|      43.2|
|         North Beach| 40.933334|
|            Presidio| 38.816666|
+--------------------+----------+
only showing top 20 rows



6. What was the average response time for fire calls in each neighborhood in 2018?

In [16]:
sf_6=fi_2018.groupBy(sf_2["Neighborhood"]).agg(round(avg(sf_2["Delay"]),4)).withColumnRenamed("round(avg(Delay), 4)","Average response")
sf_6.show()

+--------------------+----------------+
|        Neighborhood|Average response|
+--------------------+----------------+
|        Inner Sunset|          4.4381|
|      Haight Ashbury|          4.2664|
|        Lincoln Park|          3.3111|
|           Japantown|          3.3477|
|                None|          3.3633|
|         North Beach|          3.8892|
|   Lone Mountain/USF|          3.2472|
|    Western Addition|          3.2843|
|      Bernal Heights|          3.8465|
|         Mission Bay|          3.8549|
|        Hayes Valley|          3.3704|
|Financial Distric...|          4.3441|
|           Lakeshore|          3.8816|
|Bayview Hunters P...|          4.6206|
|   Visitacion Valley|          3.4838|
|      Inner Richmond|          4.3647|
|            Nob Hill|             3.8|
|Oceanview/Merced/...|          3.9472|
|      Outer Richmond|          3.6481|
|     Treasure Island|          5.4537|
+--------------------+----------------+
only showing top 20 rows



7. How many fire calls were handled by ALS units versus non-ALS units in 2018?/

In [17]:
sf_7=fi_2018.groupBy(sf_2["ALSUnit"]).count().show()

+-------+-----+
|ALSUnit|count|
+-------+-----+
|   true| 6676|
|  false| 3460|
+-------+-----+



8. What were all the different unit types?

In [18]:
sf_8=sf_df.groupBy(sf_2["UnitType"]).count()
unit_sf=sf_8.orderBy("count",ascending=False).show()

+--------------+-----+
|      UnitType|count|
+--------------+-----+
|        ENGINE|68839|
|         MEDIC|55191|
|         TRUCK|18973|
|         CHIEF|13036|
|       PRIVATE|10956|
|RESCUE CAPTAIN| 4221|
|  RESCUE SQUAD| 2952|
|       SUPPORT|  619|
|       AIRPORT|  358|
| INVESTIGATION|  151|
+--------------+-----+



9. Is there a correlation between neighborhood, zip code, and number of fire calls

In [19]:
fire_calls_count = sf_2.groupBy("Neighborhood", "Zipcode").count()
fire_calls_count.show()

+--------------------+-------+-----+
|        Neighborhood|Zipcode|count|
+--------------------+-------+-----+
|        Inner Sunset|  94122| 2161|
|Bayview Hunters P...|  94124| 9150|
|        Inner Sunset|  94114|   20|
|  West of Twin Peaks|  94112|  760|
|      Haight Ashbury|  94114|   21|
|           Glen Park|  94110|   25|
|           Excelsior|  94112| 3237|
|        Russian Hill|  94109| 2261|
|                None|  94124|    7|
|           Chinatown|  94133| 1861|
|     Pacific Heights|  94115| 2100|
|Oceanview/Merced/...|  94127|   12|
|        Potrero Hill|  94103|    5|
|        Inner Sunset|  94117|  224|
|    Golden Gate Park|  94117|  107|
|                None|   NULL|  141|
|          Noe Valley|  94131|  763|
|    Western Addition|  94117|  315|
|        McLaren Park|  94112|   36|
|      Outer Richmond|  94121| 4121|
+--------------------+-------+-----+
only showing top 20 rows



10. Which year had the most fire calls?

In [20]:
years1= sf_2.groupBy(sf_2["CallYear"]).count()
finals=years1.withColumnRenamed("count","Totalcalls").orderBy("Totalcalls",ascendeing=True)
finals.show()

+--------+----------+
|CallYear|Totalcalls|
+--------+----------+
|    2000|      5459|
|    2001|      7713|
|    2002|      8090|
|    2006|      8174|
|    2007|      8255|
|    2005|      8282|
|    2004|      8283|
|    2003|      8499|
|    2009|      8789|
|    2008|      8869|
|    2010|      9341|
|    2012|      9674|
|    2011|      9735|
|    2013|     10020|
|    2018|     10136|
|    2014|     10775|
|    2015|     11458|
|    2016|     11609|
|    2017|     12135|
+--------+----------+



11. how many num alarms is called during fire inciddent?

In [21]:
sf_11=sf_df.groupBy(sf_2["NumAlarms"]).count().show()

+---------+------+
|NumAlarms| count|
+---------+------+
|        1|174718|
|        3|   121|
|        5|    17|
|        4|    62|
|        2|   378|
+---------+------+



11. What is the trend in the number of non-life-threatening fire calls over the years?

In [26]:
sf_14=sf_2.filter(sf_df["CallTypeGroup"]=="Non Life-threatening")
sf_ca=sf_14.groupBy(sf_2["CallTypeGroup"],sf_2["CallYear"]).count().show()

+--------------------+--------+-----+
|       CallTypeGroup|CallYear|count|
+--------------------+--------+-----+
|Non Life-threatening|    2012| 2149|
|Non Life-threatening|    2013| 2352|
|Non Life-threatening|    2014| 2442|
|Non Life-threatening|    2017| 2970|
|Non Life-threatening|    2018| 2623|
|Non Life-threatening|    2015| 2638|
|Non Life-threatening|    2016| 2846|
+--------------------+--------+-----+



12. How can we use SQL tables to store this data and read it back?


In [28]:
sf_2.createOrReplaceTempView("sf_2")

In [32]:
result = Spark.sql("SELECT * FROM sf_2 WHERE CallYear = 2018")
result.toPandas()

,CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,...,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay,CallYear
0,180190613,E10,18007977,Structure Fire,01/19/2018,01/18/2018,Fire,01/19/2018 07:40:18 AM,3200 Block of CLAY ST,San Francisco,...,1,ENGINE,1,4,2,Presidio Heights,"(37.789214135744714, -122.44646398918253)",180190613-E10,2.883333,2018
1,180190640,T08,18007978,HazMat,01/19/2018,01/18/2018,Fire,01/19/2018 07:58:57 AM,200 Block of KING ST,San Francisco,...,1,TRUCK,2,3,6,Mission Bay,"(37.77732776352611, -122.39308855968541)",180190640-T08,6.333334,2018
2,180190750,T02,18007983,Alarms,01/19/2018,01/19/2018,Fire,01/19/2018 08:33:14 AM,900 Block of CLAY ST,San Francisco,...,1,TRUCK,2,1,3,Chinatown,"(37.79406255571054, -122.40832278818522)",180190750-T02,2.650000,2018
3,180190755,E08,18007984,Medical Incident,01/19/2018,01/19/2018,Code 2 Transport,01/19/2018 08:45:01 AM,600 Block of 2ND ST,San Francisco,...,1,ENGINE,2,3,6,Financial District/South Beach,"(37.78118776896415, -122.3913514497218)",180190755-E08,3.533333,2018
4,180190868,65,18007989,Medical Incident,01/19/2018,01/19/2018,Code 2 Transport,01/19/2018 09:48:53 AM,100 Block of GOLDEN GATE AVE,San Francisco,...,1,MEDIC,1,2,6,Tenderloin,"(37.78202243717766, -122.4130541482534)",180190868-65,1.100000,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10131,183034235,T08,18127270,Structure Fire,10/30/2018,10/30/2018,Fire,10/30/2018 10:04:27 PM,700 Block of LONG BRIDGE ST,San Francisco,...,1,TRUCK,3,3,6,Mission Bay,"(37.77139049567131, -122.39416141919062)",183034235-T08,2.483333,2018
10132,183034238,86,18127271,Medical Incident,10/30/2018,10/30/2018,Code 2 Transport,10/30/2018 11:18:37 PM,300 Block of WILLIAMS AVE,San Francisco,...,1,MEDIC,2,10,10,Bayview Hunters Point,"(37.73017420329081, -122.39943089033613)",183034238-86,1.950000,2018
10133,183034268,E08,18127272,Medical Incident,10/30/2018,10/30/2018,Code 2 Transport,10/30/2018 10:26:01 PM,1100 Block of 4TH ST,San Francisco,...,1,ENGINE,1,3,6,Mission Bay,"(37.77389059449051, -122.3915765620042)",183034268-E08,4.416666,2018
10134,183034485,KM03,18127291,Medical Incident,10/30/2018,10/30/2018,Code 2 Transport,10/31/2018 12:52:43 AM,800 Block of HAIGHT ST,San Francisco,...,1,PRIVATE,1,5,5,Haight Ashbury,"(37.77131921762313, -122.43619838315635)",183034485-KM03,3.566667,2018
